1. http://aiden.nibali.org/blog/2017-01-18-mode-collapse-gans/

In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.intra_op_parallelism_threads = 44
config.inter_op_parallelism_threads = 44
sess = tf.Session(config=config)

from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


In [2]:
import numpy as np

from keras.layers import Input
from keras.layers.core import Activation
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

# Discriminator
D = Sequential()
D.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), input_shape=(28, 28, 1), padding='same', kernel_initializer=initializers.RandomNormal(stddev=0.02)))
D.add(LeakyReLU(0.2))
D.add(Dropout(0.3))
D.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), input_shape=(28, 28, 1), padding='same'))
D.add(LeakyReLU(0.2))
D.add(Dropout(0.3))
D.add(Flatten())
D.add(Dense(1))
D.add(Activation('sigmoid'))
D.summary()

# compile
optimizer = Adam(lr=0.0002, beta_1=0.5)
D.compile(
    loss='binary_crossentropy', 
    optimizer=optimizer
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          102464    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
__________

In [3]:
# Generator
G = Sequential()
G.add(Dense(input_dim=100, output_dim=128*7*7, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
G.add(LeakyReLU(0.2))
G.add(Reshape((7, 7, 128)))
G.add(UpSampling2D(size=(2, 2)))
G.add(Conv2D(64, kernel_size=(5, 5), padding='same'))
G.add(LeakyReLU(0.2))
G.add(UpSampling2D(size=(2, 2)))
G.add(Conv2D(1, (5, 5), padding='same'))
G.add(Activation('tanh'))
G.summary()

# compile
G.compile(
    loss='binary_crossentropy', 
    optimizer=optimizer,
    metrics=['accuracy']
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6272)              633472    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 6272)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        204864    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 64)        0         
__________

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6272, kernel_initializer=<keras.ini..., input_dim=100)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# compile GAN
D.trainable = False
GAN = Sequential()
GAN.add(G)
GAN.add(D)
GAN.compile(
    loss='binary_crossentropy', 
    optimizer=optimizer
)
D.trainable = True

In [ ]:
from keras.datasets import mnist
import matplotlib.pyplot as plt

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train[:, :, :, None]
X_test = X_test[:, :, :, None]

EPOCH = 50
BATCH_SIZE = 256

for EPOCH in range(100):
    print("Epoch is", EPOCH)
    print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
    for index in range(int(X_train.shape[0]/BATCH_SIZE)):
        # Get a random set of input noise and images
        noise = np.random.normal(0, 1, size=[BATCH_SIZE, 100])
        imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=BATCH_SIZE)]
        # Generate fake MNIST images
        generatedImages = G.predict(noise)
        X = np.concatenate([imageBatch, generatedImages])
        # Labels for generated and real data
        yDis = np.zeros(2*BATCH_SIZE)
        # One-sided label smoothing from improved WGAN
        yDis[:BATCH_SIZE] = 0.9
        # Train discriminator
        D.trainable = True
        d_loss = D.train_on_batch(X, yDis)
        # Train generator
        noise = np.random.normal(0, 1, size=[BATCH_SIZE, 100])
        yGen = np.ones(BATCH_SIZE)
        D.trainable = False
        g_loss = GAN.train_on_batch(noise, yGen)   
        if index % 50 == 0:
            print("batch %d g_loss : %f, d_loss : %f" % (index, g_loss, d_loss))
        
    try_input = np.random.normal(0, 1, size=[100, 100])
    preds = G.predict(try_input)
    plt.figure(figsize=(10,10))
    for i in range(preds.shape[0]):
        plt.subplot(10, 10, i+1)
        plt.imshow(preds[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

('Epoch is', 0)
('Number of batches', 234)
batch 0 g_loss : 0.691882, d_loss : 0.690273
batch 50 g_loss : 0.754781, d_loss : 0.653503
batch 100 g_loss : 0.857651, d_loss : 0.664915
